<a href="https://colab.research.google.com/github/YNNJN/colorPOOL_Data/blob/master/%EA%B8%B0%EC%A4%80%ED%96%89%EB%A0%AC%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 기준행렬 K 생성
- 기준 행렬이란 조화관계 예측 위한 기준 데이터임
- 즉 각 테마가 어떤 색채를 포함하는지를 나타내는 색채-테마 관계 기준 행렬임
- N X M 행렬
    - N
        - 표본 색채 수(8396)
    - M
        - 테마 수(3000)
    - value
        - 표본 색채가(행)이 테마(열)에 포함될 경우 해당 테마의 rating 값
        - 포함되지 않을 경우 0

In [ ]:
!git clone https://github.com/YNNJN/colorPool_Data.git

Cloning into 'colorPool_Data'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 52 (delta 26), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [ ]:
!ls -ltr

total 8
drwxr-xr-x 1 root root 4096 Sep 16 16:29 sample_data
drwxr-xr-x 4 root root 4096 Sep 22 23:23 colorPool_Data


In [ ]:
# connect to google drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

data = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/quantize.csv')
data = data.drop([data.columns[0]], axis=1)
data

,L,a,b
0,36,18,30
1,6,3,3
2,72,33,81
3,42,69,63
4,99,0,36
...,...,...,...
14995,54,-18,-3
14996,48,-3,15
14997,57,-24,0
14998,90,0,57


## 1) 표본 색채 (row)

In [ ]:
# 표본 색채 설정 위해 중복되는 컬러 삭제
# 결과 : 8396개 컬러

colorData = data.drop_duplicates()
colorData = colorData.reset_index(drop=True)
colorData

,L,a,b
0,36,18,30
1,6,3,3
2,72,33,81
3,42,69,63
4,99,0,36
...,...,...,...
8391,45,72,66
8392,84,24,66
8393,72,24,84
8394,93,-15,3


## 2) 표본 배색 (column)

In [ ]:
# 데이터프레임 형태 변환 15000*3 => 3000*15

df1 = data[:3000].reset_index(drop=True).rename(columns={"L":'L1',"a":'a1',"b":'b1'})
df2 = data[3000:6000].reset_index(drop=True).rename(columns={"L":'L2',"a":'a2',"b":'b2'})
df3 = data[6000:9000].reset_index(drop=True).rename(columns={"L":'L3',"a":'a3',"b":'b3'})
df4 = data[9000:12000].reset_index(drop=True).rename(columns={"L":'L4',"a":'a4',"b":'b4'})
df5 = data[12000:15000].reset_index(drop=True).rename(columns={"L":'L5',"a":'a5',"b":'b5'})

df_all = pd.concat([df1,df2,df3,df4,df5], axis=1).reset_index(drop=True)
df_all

,L1,a1,b1,L2,a2,b2,L3,a3,b3,L4,a4,b4,L5,a5,b5
0,36,18,30,57,6,48,51,0,27,30,0,21,36,9,30
1,6,3,3,96,3,18,87,-6,30,84,-15,36,84,-21,42
2,72,33,81,57,54,60,75,-15,81,63,-30,66,57,-9,-3
3,42,69,63,54,-6,-24,48,75,0,33,18,33,0,0,0
4,99,0,36,81,6,24,63,45,60,48,39,45,30,24,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,54,54,12,99,-6,66,96,-24,24,75,-36,27,54,-18,-3
2996,60,6,69,96,3,18,93,-3,72,18,0,3,48,-3,15
2997,48,75,66,60,45,72,75,30,84,66,-24,69,57,-24,0
2998,6,18,9,33,45,48,60,45,63,72,36,69,90,0,57


## 3) rating 데이터 (value)

In [ ]:
data = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/matrixData.csv')

ratingData = data.loc[:, ['rating']]
idData = data.loc[:, ['id']]
nameData = data.loc[:, ['name']]

matrixData = pd.concat([df_all, ratingData], axis=1)
matrixData

,L1,a1,b1,L2,a2,b2,L3,a3,b3,L4,a4,b4,L5,a5,b5,rating
0,36,18,30,57,6,48,51,0,27,30,0,21,36,9,30,5.0
1,6,3,3,96,3,18,87,-6,30,84,-15,36,84,-21,42,5.0
2,72,33,81,57,54,60,75,-15,81,63,-30,66,57,-9,-3,5.0
3,42,69,63,54,-6,-24,48,75,0,33,18,33,0,0,0,5.0
4,99,0,36,81,6,24,63,45,60,48,39,45,30,24,30,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,54,54,12,99,-6,66,96,-24,24,75,-36,27,54,-18,-3,4.0
2996,60,6,69,96,3,18,93,-3,72,18,0,3,48,-3,15,4.0
2997,48,75,66,60,45,72,75,30,84,66,-24,69,57,-24,0,4.0
2998,6,18,9,33,45,48,60,45,63,72,36,69,90,0,57,4.0


## 4) 기준 행렬 생성
- 색채-테마 관계 기준 행렬
    - row : 표본 색채를
    - column : 테마(배색)
- 테마가 어떤 색채를 포함하는지를 나타냄
    - 포함하는 경우 rating의 value를 가짐
- sparse matrix 이므로
    - item based collaborative filtering을 통해
    - 전체 색과 테마 색의 유사도를 이용해 matrix의 빈칸을 채워야 함

In [ ]:
result = [[] for _ in range(8396)]

for i in range(len(colorData)):
    for j in range(len(matrixData)):
        rating = 0
        for k in range(5):
            if colorData['L'][i] == matrixData.iloc[j,3*k] and colorData['a'][i] == matrixData.iloc[j, 3*k+1] and colorData['b'][i] == matrixData.iloc[j,3*k+2]:
                rating = matrixData['rating'][j]
                break
        result[i].append(rating)

In [ ]:
# 컬럼 명 설정을 위해 nameData의 type :  DF -> list 변환

id = list(np.array(idData['id'].tolist()))

In [ ]:
matrix = pd.DataFrame(data=result, columns=id)
matrix

,872770.0,642440.0,619900.0,41437.0,41351.0,624320.0,628840.0,634440.0,635260.0,636160.0,651270.0,613430.0,658640.0,662550.0,666100.0,667970.0,668180.0,668560.0,670440.0,670550.0,617030.0,610670.0,672220.0,544090.0,418900.0,428400.0,439050.0,441740.0,473180.0,482920.0,487800.0,532540.0,545180.0,601960.0,550730.0,556020.0,560440.0,580810.0,596640.0,596970.0,...,516760.0,843460.0,843360.0,517580.0,517720.0,527300.0,502210.0,844240.0,844050.0,486340.0,530830.0,844160.0,844150.0,867420.0,488620.0,844020.0,485580.0,489590.0,489890.0,489930.0,490100.0,529960.0,491740.0,486260.0,485530.0,492210.0,843130.0,478680.0,843100.0,481050.0,482300.0,482350.0,482900.0,867420.0,484530.0,483060.0,483270.0,484000.0,484050.0,484080.0
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
matrix.to_csv('gdrive/My Drive/colorPOOL/kulerData/referenceMatrix.csv')